# D-REX

Minimal procedure:
1. Set input sequence
2. Calculate prior, given
    1. an input sequence to use
    2. parameters, including
        1. distribution (default: gaussian, options: gaussian, lognormal, gmm, poisson)
        2. D (default: 1, 50 for poisson)
3. Add prior to parameters, and optionally set further parameters
    1. hazard (default: 0.01)
    2. obsnz (default: 0.0)
    3. memory (default: inf)
    4. maxhyp (default: inf)
    5. Undocumented: predscale (default: 0.001)
4. Run model, optionally:
    1. Calculate belief dynamics (post_DREX_beliefdynamics.m)
    2. Calculate change decisions (post_DREX_changedecision.m)
    3. Calculate predictive distribution (post_DREX_prediction.m)
6. Interpret results

# CMME interna: IO paths

In [ ]:
from cmme.model_io import *

#io_paths = ModelIO("./io", "general_procedure").generate_drex_paths()
io_paths = {'base_directory': '/Users/alexander/git/cmme-jupyter/io/drex/', 'generic_filename_prefix': '/Users/alexander/git/cmme-jupyter/io/drex/general_procedure_', 'input_file_path': '/Users/alexander/git/cmme-jupyter/io/drex/general_procedure_input', 'output_file_path': '/Users/alexander/git/cmme-jupyter/io/drex/general_procedure_output'}

print(io_paths)

# 1. Set input sequence

In [ ]:
x1 = [1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1]
x2 = [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
x3 = [1, 2, 3, 3, 2, 1, 1, 2, 3, 3, 2, 1]

In [ ]:
# a) Single-feature input, e.g.
input_sequence = x1

In [ ]:
# b) Multi-feature input, e.g.
input_sequence = [x1, x2, x3]

In [ ]:
# Then:
print(input_sequence)

# 2. Calculate prior

In [ ]:
from cmme.drex import *

## Choose distribution type and parameters

In [ ]:
# a) Gaussian
distribution = DREXDistribution.GAUSSIAN

# Temporal dependency, >= 1
D = 4

In [ ]:
# b) Lognormal
distribution = DREXDistribution.LOGNORMAL

# Temporal dependency, >= 1
D = 1

In [ ]:
# c) GMM (Gaussian Mixture Model)
distribution = DREXDistribution.GMM

# Temporal dependency, allowed value: 1
D = 1

# Max. number of components, >= 1
max_ncomp = 1

In [ ]:
# d) Poisson
distribution = DREXDistribution.POISSON

# D (here: "interval size"), >= 1
D = 1

## Choose input sequence (sample)

In [ ]:
# Note: For simplicity, use the complete input sequence
prior_input_sequence = input_sequence
print(prior_input_sequence)

## Calculate prior

In [ ]:
# a) If Gaussian, Lognormal, or Poisson
prior_estimator = DREXPriorEstimator(io_paths).estimate(distribution, prior_input_sequence, D)

In [ ]:
# b) If GMM
prior_estimator = DREXPriorEstimator(io_paths).estimate(distribution, prior_input_sequence, D, max_ncomp)

In [ ]:
# Then:
print(prior_estimator)

## Parse prior

In [ ]:
file_path = prior_estimator["result"]["output_file_path"]

prior = DREXPriorOutputParameters.from_mat(file_path)
print(prior)

# 3. Add prior to model / Build model

In [ ]:
builder = DREXInstanceBuilder(io_paths, prior)

In [ ]:
instance = builder.build()

# 4. Run model

In [ ]:
drex_result = instance.observe(input_sequence)
print(drex_result)

# 5. Interpret

In [ ]:
output_file_path = drex_result["result"]["output_file_path"]

In [ ]:
# ToDo D-REX plot